In [1]:
import json, requests, uuid
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

In [ ]:
cognitive_services_key="YOUR_KEY"
cognitive_services_endpoint="URL"
translator_endpoint = "https://api.cognitive.microsofttranslator.com/"
translator_key = "YOUR_KEY"
translator_region = "westeurope"

In [2]:
credential = AzureKeyCredential(cognitive_services_key)
text_analytics_client = TextAnalyticsClient(endpoint=cognitive_services_endpoint, credential=credential)

In [3]:
# Code to call Text Analytics service to analyze sentiment in text
file_name = "./data/tweet3.json"

# Language Detection
with open(file_name, mode='rt') as file:
    content = json.load(file)

Tweet text

In [4]:
content['TweetText']

'RT @TgLa7: #Nato: stop Ungheria su ingresso della Svezia e Finlandia'

Language detection

In [5]:
detected_languages = text_analytics_client.detect_language([content['TweetText']])
iso_lang = detected_languages[0].primary_language.iso6391_name

Sentiment analysis

In [10]:
text_analytics_client.analyze_sentiment([content['TweetText']])

[AnalyzeSentimentResult(id=0, sentiment=neutral, warnings=[], statistics=None, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.94, negative=0.05), sentences=[SentenceSentiment(text=RT @TgLa7: #Nato: stop Ungheria su ingresso della Svezia e Finlandia, sentiment=neutral, confidence_scores=SentimentConfidenceScores(positive=0.01, neutral=0.94, negative=0.05), length=68, offset=0, mined_opinions=[])], is_error=False)]

Finding Key Phrases

In [40]:
text_analytics_client.extract_key_phrases([content['TweetText']], language=iso_lang)

[ExtractKeyPhrasesResult(id=0, key_phrases=['RT', 'stop', 'Ungheria', 'ingresso', 'Svezia', 'Finlandia'], warnings=[], statistics=None, is_error=False)]

Translation

In [11]:

# If you encounter any issues with the base_url or path, make sure
# that you are using the latest endpoint: https://docs.microsoft.com/azure/cognitive-services/translator/reference/v3-0-translate
path = '/translate?api-version=3.0'
params = f'&from={iso_lang}&to=en'
constructed_url = translator_endpoint + path + params

headers = {
    'Ocp-Apim-Subscription-Key': translator_key,
    'Ocp-Apim-Subscription-Region': translator_region,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

# You can pass more than one object in body.
body = [{
    'text' : content['TweetText']
}]
request = requests.post(constructed_url, headers=headers, json=body)
response = request.json()

print(json.dumps(response, sort_keys=True, indent=4, separators=(',', ': ')))

[
    {
        "translations": [
            {
                "text": "RT @TgLa7: #Nato: stop Hungary on entry of Sweden and Finland",
                "to": "en"
            }
        ]
    }
]
